# Effects em Python - Uma abordagem funcional

Este notebook explora o conceito de **Effects** em programação funcional usando Python. 

## O que são Effects?

Um `Effect` é essencialmente um **thunk** - uma função que não recebe argumentos e retorna um valor quando chamada. Esta é uma abstração poderosa que nos permite separar a **descrição** de uma computação da sua **execução**.

### Por que isso é importante?

Em programação imperativa tradicional, quando escrevemos:
```python
result = expensive_operation()
```

A operação é executada **imediatamente**. Com Effects, podemos descrever a operação sem executá-la:
```python
effect = lambda: expensive_operation()  # Descrição da operação
result = effect()  # Execução quando necessário
```

### Vantagens dos Effects:

1. **Controle de Execução**: Decidimos **quando** e **quantas vezes** executar
2. **Composição**: Podemos combinar Effects de forma elegante
3. **Testabilidade**: Easier testing através de injeção de dependência
4. **Lazy Evaluation**: Computações só acontecem quando necessário
5. **Error Handling**: Tratamento de erros mais expressivo e controlado

Isso nos permite construir sistemas mais robustos, testáveis e compostos através de abstrações funcionais poderosas.

In [105]:
from typing import Callable

type Effect[A] = Callable[[], A]

In [106]:
import random

# Isso é uma função normal, ela retorna uma int
def random_number() -> int:
    return random.randint(1, 100)

# Essa é uma função que retorna um `Effect` de int
# Ou seja, ela não executa a função imediatamente, mas retorna uma função que, 
# quando chamada, executará a função original e retornará o resultado.
def random_number_function() -> Effect[int]:
  return random_number

# Mesma coisa que
# random_number_function: lambda: random_number()

# Vamos criar um helper que simplesmente roda o `Effect` e imprime o resultado
def run[A](effect: Effect[A]) -> None:
    print(effect())

run(random_number_function())

50


## Por que usar Effects?

**Porque agora temos uma função que pode ser executada mais tarde!**

### O Poder da Composição

Em vez de modificar o resultado diretamente, conseguimos modificar a **função em si**. Isso abre possibilidades incríveis:

1. **Repetição Controlada**: Podemos executar uma operação múltiplas vezes
2. **Transformação de Comportamento**: Adicionar funcionalidades como retry, timeout, logging
3. **Composição Declarativa**: Construir operações complexas a partir de simples

### Exemplo Prático: Repeat

Note que `repeat` não executa a função imediatamente - ela retorna um **novo Effect** que, quando executado, retorna uma lista de resultados. Isso é fundamental: Effects sempre retornam Effects, permitindo composição infinita.

### Lazy Evaluation em Ação

```python
# Sem Effects - execução imediata
numbers = [random_number() for _ in range(5)]  # Executa AGORA

# Com Effects - execução controlada  
repeat_effect = repeat(random_number_function(), 5)  # Apenas descrição
numbers = repeat_effect()  # Executa QUANDO quisermos
```

Esta separação entre descrição e execução é a essência dos Effects e permite que construamos sistemas mais flexíveis e controlados.

In [107]:
def repeat[A](effect: Effect[A], times: int) -> Effect[list[A]]:
    def repeated_effect() -> list[A]:
        results: list[A] = []
        for _ in range(times):
            result = effect()
            results.append(result)
        return results

    return repeated_effect

# Agora podemos fazer composição de efeitos
# Podemos repetir a execução de `random_number_function` 5 vezes
run(repeat(random_number_function(), 5))

[100, 72, 78, 35, 8]


## A Magia da Lazy Evaluation

Note que até eu fazer `run`, **nenhuma computação acontece**. Estamos apenas construindo uma "receita" de como executar as operações.

### Reutilização e Flexibilidade

Como `repeat` retorna um novo `Effect`, posso:
- **Reutilizar** a mesma "receita" múltiplas vezes
- **Compor** com outras operações
- **Modificar** o comportamento sem alterar o código original

### Benefícios Práticos:

1. **Performance**: Computações caras só acontecem quando necessário
2. **Debugging**: Posso inspecionar a "receita" antes de executar
3. **Testing**: Posso substituir Effects por mocks facilmente
4. **Caching**: Posso implementar cache ao nível do Effect

Isso é especialmente poderoso quando lidamos com operações que podem falhar, como chamadas de rede, I/O ou operações custosas.

In [120]:
twice = repeat(random_number_function(), 2)
thrice = repeat(random_number_function(), 3)
twice_twice = repeat(twice, 2)

run(twice)
run(thrice)
run(twice_twice)

[24, 81]
[46, 48, 8]
[[29, 35], [39, 12]]


## Lidando com Erros de Forma Funcional

O tratamento de erros em programação funcional é fundamentalmente diferente da abordagem imperativa. Em vez de usar try-catch em todo lugar, construímos Effects que **encapsulam** a possibilidade de falha.

### Filosofia dos Effects com Erros:

1. **Erros são Valores**: Tratamos falhas como parte normal do fluxo
2. **Composição Segura**: Effects podem ser compostos mesmo quando podem falhar
3. **Controle Explícito**: Decidimos onde e como tratar erros

### Benefícios desta Abordagem:

- **Previsibilidade**: Sabemos exatamente onde erros podem ocorrer
- **Composição**: Podemos adicionar retry, fallbacks, logging de forma declarativa
- **Separação de Responsabilidades**: Lógica de negócio separada do tratamento de erros

**Lembre-se:** o `thunk` (nossa função Effect) não possui argumentos, mas uma função pode receber argumentos e retornar um `Effect` que **captura** esses argumentos no closure.

### Exemplo Prático:
```python
# Em vez de:
try:
    result = risky_operation(param)
    process(result)
except Exception:
    handle_error()

# Usamos:
effect = create_risky_effect(param)
safe_effect = add_error_handling(effect)
result = safe_effect()
```

In [126]:
def fail_if_small_number(number: Effect[int]) -> Effect[int | Exception]:
    def effect() -> int | Exception:
        n = number()
        if n < 70:
            return Exception("Number is too small")
        return n

    return effect

# Não precisa de try/catch
run(fail_if_small_number(random_number_function()))


Number is too small


## Padrão Retry - Construindo Resiliência

O erro só acontece **DEPOIS** de chamar `run`. Isso é fundamental! Conseguimos construir toda a lógica de retry **antes** de qualquer execução acontecer.

### Por que Retry é Poderoso com Effects?

1. **Composição Natural**: Retry é apenas outro Effect que wrapa outros Effects
2. **Configuração Declarativa**: Podemos definir estratégias complexas de retry
3. **Transparência**: O Effect resultante ainda é um Effect normal

### Estratégias de Retry Avançadas:

Com Effects, podemos facilmente implementar:
- **Exponential Backoff**: Aumentar tempo entre tentativas
- **Circuit Breaker**: Parar tentativas após muitas falhas
- **Conditional Retry**: Retry apenas para certos tipos de erro
- **Jittered Retry**: Adicionar randomness para evitar thundering herd

Vamos criar um efeito que tenta executar um efeito várias vezes até ter sucesso:

In [110]:
def retry[A](effect: Effect[A], times: int) -> Effect[A | Exception]:
  def repeated_effect():
      for _ in range(times):
          result = effect()
          if isinstance(result, Exception):
              print("Effect failed, retrying...")
              continue
          return result
      return Exception("Effect failed after retries")
  return repeated_effect

# Vamos tentar executar o efeito que falha 3 vezes
run(retry(fail_if_small_number(random_number_function()), 3))

Effect failed, retrying...
Effect failed, retrying...
Effect failed, retrying...
Effect failed after retries


### Observando o Comportamento do Retry

Execute a célula acima algumas vezes e observe como o retry funciona na prática. Você verá:

1. **Tentativas Múltiplas**: O Effect tenta até 3 vezes antes de desistir
2. **Feedback Visual**: Cada falha é logada, mostrando o processo
3. **Composição Transparente**: O resultado final ainda é um Effect simples

Este padrão é extremamente comum em sistemas distribuídos e APIs, onde falhas temporárias são esperadas.

## Padrão OrElse - Graceful Degradation

Mas as vezes o erro não é o fim do mundo, e tem um valor que podemos utilizar.

O padrão `orElse` implementa **graceful degradation** - quando algo falha, fornecemos um valor padrão sensato em vez de quebrar todo o sistema.

### Filosofia do Graceful Degradation:

1. **Sistemas Resilientes**: Preferem funcionar parcialmente a não funcionar
2. **Experiência do Usuário**: Melhor mostrar dados parciais que uma tela de erro
3. **Disponibilidade**: Mantém o sistema funcionando mesmo com falhas

### Estratégias de Fallback:

- **Valor Padrão**: Como mostrado no exemplo (retorna 0)
- **Cache**: Usar dados antigos quando novos não estão disponíveis  
- **Serviço Alternativo**: Tentar outro endpoint/serviço
- **Dados Estáticos**: Usar configurações fixas como último recurso

### Composição de Padrões de Resiliência:

**SEMPRE, SEMPRE** retornamos um `Effect`. Isso permite que todas as funções sejam compostas e encadeadas, criando pipelines de resiliência:

```
Original Effect -> Retry -> OrElse -> Telemetry -> Execute
```

Cada etapa adiciona uma camada de robustez sem quebrar a composição.

In [129]:
def or_else[A](effect: Effect[A], fallback: A) -> Effect[A]:
    def wrapped_effect() -> A:
        result = effect()
        if isinstance(result, Exception):
            print("Effect failed, returning fallback value:", fallback)
            return fallback
        print("Effect succeeded with value:", result)
        return result
    return wrapped_effect

# Se falhar mesmo depois de tentar, vira 0
retry_if_small = retry(fail_if_small_number(random_number_function()), 2)
run(repeat(or_else(retry_if_small, 0), 20))

Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect succeeded with value: 98
Effect succeeded with value: 96
Effect succeeded with value: 82
Effect succeeded with value: 93
Effect succeeded with value: 96
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect failed, retrying...
Effect succeeded with value: 100
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect succeeded with value: 72
Effect succeeded with value: 86
Effect failed, retrying...
Effect fai

## Telemetria e Observabilidade - Monitoramento Transparente

O legal é que como temos controle de **QUANDO** executar o efeito, podemos adicionar funcionalidades de monitoramento **antes**, **durante** e **depois** da execução, sem modificar a lógica original.

### Observabilidade em Sistemas Modernos:

1. **Logs**: Registro de eventos para debugging
2. **Métricas**: Medições quantitativas (latência, throughput, erro rate)
3. **Tracing**: Acompanhamento de requisições através do sistema
4. **Alerting**: Notificações quando algo está errado

### Vantagens dos Effects para Observabilidade:

- **Transparência**: Adicionar telemetria sem modificar código de negócio
- **Composição**: Combinar múltiplos tipos de observabilidade
- **Configuração**: Facilmente ligar/desligar em diferentes ambientes
- **Testing**: Pode ser mockado ou desabilitado em testes

### O Princípio da Transparência

**Fundamental:** Funções de observabilidade como `log_effect` são **transparentes** - elas **observam** o valor mas **não o modificam**. Isso significa:

```python
# O resultado é EXATAMENTE o mesmo
original_effect = random_number_function()
logged_effect = log_effect(original_effect, "debug")

# Ambos retornam o mesmo valor, mas logged_effect adiciona logs
```

**Por que isso é importante:**
- **Comportamento Preservado**: A lógica de negócio permanece intacta
- **Composição Segura**: Pode ser adicionado/removido sem quebrar o código
- **Testing**: Logs podem ser facilmente desabilitados em testes
- **Performance**: Zero overhead quando logging está desabilitado

Cada wrapper adiciona uma camada de observabilidade mantendo a interface Effect consistente.

In [137]:
def log_effect[A](effect: Effect[A], text: str) -> Effect[A]:
    def wrapped_effect() -> A:
        print(f"[--> Starting effect: {text}")
        result = effect()
        print(f"[--> End effect: {text}")
        return result
    return wrapped_effect

operation = or_else(retry(fail_if_small_number(random_number_function()), 2), 0)

run(repeat(log_effect(operation, "Repeat Effect"), 10))

[--> Starting effect: Repeat Effect
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
[--> End effect: Repeat Effect
[--> Starting effect: Repeat Effect
Effect failed, retrying...
Effect succeeded with value: 95
[--> End effect: Repeat Effect
[--> Starting effect: Repeat Effect
Effect succeeded with value: 88
[--> End effect: Repeat Effect
[--> Starting effect: Repeat Effect
Effect failed, retrying...
Effect succeeded with value: 97
[--> End effect: Repeat Effect
[--> Starting effect: Repeat Effect
Effect succeeded with value: 80
[--> End effect: Repeat Effect
[--> Starting effect: Repeat Effect
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
[--> End effect: Repeat Effect
[--> Starting effect: Repeat Effect
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
[--> End effect: Repeat Effect
[--> Starting effect: Repeat Effect
Effect failed, retryin

## Transformação de Dados com Effects

O Effect não precisa sempre retornar o mesmo tipo `A` - podemos **transformar** o resultado durante a execução.

### Exemplo: Medição de Performance

Aqui criamos um Effect que retorna não apenas o resultado original, mas **também** quanto tempo a operação demorou. Isso é útil para:

1. **Performance Monitoring**: Detectar operações lentas
2. **SLA Tracking**: Verificar se estamos dentro dos tempos esperados  
3. **Optimization**: Identificar gargalos de performance
4. **Alerting**: Disparar alertas quando operações demoram muito


In [140]:
from typing import Tuple
from time import perf_counter

def timed[A](effect: Effect[A]) -> Effect[Tuple[A, float]]:
    def timed_effect() -> Tuple[A, float]:
        start = perf_counter()
        result = effect()
        end = perf_counter()
        return (result, (end - start) * 10000)

    return timed_effect

operation = timed(or_else(retry(fail_if_small_number(random_number_function()), 2), 0))

run(repeat(operation, 5))

Effect succeeded with value: 83
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect succeeded with value: 76
Effect succeeded with value: 80
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
[(83, 0.6329700045171194), (0, 0.45603999751619995), (76, 0.15800000255694613), (80, 0.14637997082900256), (0, 0.2860300082829781)]


## O Problema dos Parênteses Aninhados

Vai tomar no cu, tem tanto parêntesis que eu fiquei perdido!

### O Problema da Composição Nested

Quando composição de funções fica complexa, temos vários problemas:

1. **Legibilidade**: Difícil de ler e entender o fluxo
2. **Manutenibilidade**: Difícil de adicionar/remover steps
3. **Debugging**: Complicado identificar onde erros acontecem
4. **Ordem de Execução**: Precisa ler de dentro para fora

### Composição Funcional vs Procedural

```python
# Estilo aninhado - confuso
result = h(g(f(data)))

# Estilo procedural - verboso  
# E dar nomes é um dos problemas mais difícieis na programação
temp1 = f(data)
temp2 = g(temp1)
result = h(temp2)

# Estilo pipeline - claro
result = pipe(data, f, g, h)
```

### A Inspiração UNIX

**Solução:** No UNIX a solução é encadear comandos com pipe!
- Exemplo: `ls -l | grep py | wc -l` em vez de `wc -l (grep py (ls -l))`
- Cada comando processa a saída do anterior
- Leitura da esquerda para direita (ordem natural)
- Fácil adicionar/remover steps no meio

### Pipeline em Programação Funcional

O `toolz` tem uma função chamada `pipe` que replica essa funcionalidade:
- `pipe(data, f, g, h)` === `h(g(f(data)))`
- Leitura natural da esquerda para direita
- Cada função recebe o resultado da anterior
- Composição declarativa e clara

### Por que Precisamos de Lambdas no Pipe?

O `pipe` do toolz espera que cada função receba **exatamente um argumento** (o resultado da função anterior). Mas muitas das nossas funções como `retry`, `or_else` e `repeat` precisam de argumentos adicionais:

```python
# Problema: retry precisa de 2 argumentos
retry(effect, 2)  # Effect + número de tentativas

# Solução: lambda "fixa" o segundo argumento
lambda effect: retry(effect, 2)  # Agora recebe apenas 1 argumento
```

**Por que isso funciona:**
1. **Aridade Unária**: Pipe exige funções que recebem 1 argumento
2. **Closure**: Lambda "captura" os argumentos extras (2, 0, 20, etc.)
3. **Adaptação**: Transformamos função de N argumentos em função de 1 argumento
4. **Composição**: Todas as etapas do pipeline têm a mesma interface

Sem lambdas, o pipe quebraria porque tentaria passar apenas o Effect para funções que esperam mais parâmetros.

In [133]:
from toolz import pipe # type: ignore

result = pipe( # type: ignore
    random_number_function(),# O primeiro valor do pipe é um valor de verdade  # type: ignore
    fail_if_small_number, # Nesse caso, o primeiro argumento é o Effect, então funciona 
    lambda effect: retry(effect, 2), # type: ignore
    lambda effect: or_else(effect, 0), # type: ignore
    lambda effect: repeat(effect, 20), # type: ignore
    timed, # como não precisa de outros argumentos, podemos passar diretamente
    lambda effect: log_effect(effect, "Big Operation"), # type: ignore
    run # o mesmo para o run
)

# Agora eu consigo ler de fora para dentro
# E consigo ver o que está acontecendo em cada etapa
# Como se fosse um pipeline de dados

[--> Starting effect: Big Operation
Effect succeeded with value: 95
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect succeeded with value: 89
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect succeeded with value: 91
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect failed, retrying...
Effect succeeded with value: 74
Effect failed, retrying...
Effect succeeded with value: 79
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect succeeded with value: 94
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect failed, retrying...
Effect succe

## Partial Application - Fazendo Melhor Ainda

Ainda conseguimos fazer melhor! O `pipe` é poderoso, mas temos um problema técnico.

### O Problema da Aridade

O `pipe` espera uma função que recebe **exatamente um argumento**, mas nossas funções recebem:
1. Um `Effect` como primeiro argumento  
2. Às vezes parâmetros de configuração (como `times`, `fallback`)

### Solução: Partial Application

**Técnica fundamental da programação funcional:** criar uma função que recebe alguns argumentos e retorna uma nova função que recebe o restante dos argumentos.

```python
# Função original com 2 argumentos
def add(a: int, b: int) -> int:
    return a + b

# Partial application - "fixa" o primeiro argumento
add_5 = partial(add, 5)  # Equivale a: lambda b: add(5, b)
result = add_5(3)  # result = 8
```

### Vantagens para Pipelines:

1. **Configuração Antecipada**: Definimos parâmetros uma vez
2. **Reutilização**: Mesma configuração para múltiplos casos
3. **Composição Limpa**: Todas as funções têm a mesma assinatura no pipeline
4. **Semântica Clara**: Nome da função indica claramente sua configuração

Com partial application, transformamos:
```python
lambda effect: retry(effect, 3)  # Verboso
```
Em:
```python
with_retry(3)  # Claro e reutilizável
```

In [115]:
from toolz import partial # type: ignore

def with_repeat[A](times: int) -> Callable[[Effect[A]], Effect[list[A]]]:
  return partial(repeat, times=times) # type: ignore

# with_repeat(5)(random_number_function()) == repeat(random_number_function(), 5)

def with_retry[A](times: int) -> Callable[[Effect[A]], Effect[A]]:
    return partial(retry, times=times) # type: ignore

def with_or_else[A](fallback: A) -> Callable[[Effect[A]], Effect[A]]:
    return partial(or_else, fallback=fallback) # type: ignore

def with_telemetry[A](text: str) -> Callable[[Effect[A]], Effect[A]]:
    return partial(log_effect, text=text) # type: ignore


pipe( # type: ignore
    random_number_function(), 
    with_telemetry("Random number generator"), 
    fail_if_small_number, 
    with_retry(2), 
    with_or_else(0), 
    with_repeat(20), 
    timed, 
    run
)

# Você pode ler pensando:
# Rodar esse efeito com telemetria,
# se falhar, tentar mais duas vezes,
# se falhar, retornar 0,
# repetir 20 vezes,
# medir o tempo de execução

[--> Starting effect: Random number generator
[--> End effect: Random number generator
Effect failed, retrying...
[--> Starting effect: Random number generator
[--> End effect: Random number generator
Effect failed, retrying...
Effect failed, returning fallback value: 0
[--> Starting effect: Random number generator
[--> End effect: Random number generator
Effect failed, retrying...
[--> Starting effect: Random number generator
[--> End effect: Random number generator
Effect failed, retrying...
Effect failed, returning fallback value: 0
[--> Starting effect: Random number generator
[--> End effect: Random number generator
Effect succeeded with value: 80
[--> Starting effect: Random number generator
[--> End effect: Random number generator
Effect failed, retrying...
[--> Starting effect: Random number generator
[--> End effect: Random number generator
Effect failed, retrying...
Effect failed, returning fallback value: 0
[--> Starting effect: Random number generator
[--> End effect: Rando

## Map e FlatMap - Transformações Poderosas

Duas das operações mais fundamentais em programação funcional. Elas permitem transformar valores **dentro** do contexto do Effect sem quebrar a composição.

### Map - Transformação Simples

É **LITERALMENTE** a mesma coisa que um map de list, mas para Effects:

```python
# Com listas
numbers = [1, 2, 3]
strings = map(str, numbers)  # ["1", "2", "3"]

# Com Effects  
number_effect = lambda: 42
string_effect = map_effect(number_effect, str)  # Effect[str]
```

**Importante**: Map mantém o contexto Effect. Se temos `Effect[A]` e aplicamos função `A -> B`, obtemos `Effect[B]`.

### FlatMap - Composição de Effects

E se a função que queremos aplicar também retorna um Effect? Teríamos `Effect[Effect[B]]` - um Effect aninhado!

FlatMap resolve isso "achatando" a estrutura:

```python
# Problema sem FlatMap
def duplicate_effect(x: int) -> Effect[int]:
    return lambda: x * 2

nested = map_effect(number_effect, duplicate_effect)  # Effect[Effect[int]] ❌

# Solução com FlatMap  
flattened = flat_map_effect(number_effect, duplicate_effect)  # Effect[int] ✅
```

### Casos de Uso Práticos:

- **Map**: Transformar dados (formatting, parsing, calculations)
- **FlatMap**: Composição de operações que podem falhar ou são assíncronas

In [116]:
def map_effect[A, B](effect: Effect[A], func: Callable[[A], B]) -> Effect[B]:
    def mapped_effect() -> B:
        result = effect()
        return func(result)
    return mapped_effect 

# Isso permite usar lambdas diretamente
run(map_effect(random_number_function(), lambda x: f"Número aleatório: {x}"))

def flat_map_effect[A, B](effect: Effect[A], func: Callable[[A], Effect[B]]) -> Effect[B]:
    return func(effect())  # Chama o efeito original, depois chama o efeito que a função recebeu

def duplicate_effect(effect: Effect[int]) -> Effect[int]:
    def duplicated_effect() -> int:
        return effect() * 2
    return duplicated_effect

run(flat_map_effect(random_number_function, duplicate_effect))

Número aleatório: 34
46


## Generators para Simplificar Sintaxe

Uma coisa chata da abordagem funcional pura é a necessidade de definir uma função para cada operação. Isso torna o código verboso e menos legível.

### Solução: Generator-Based Effects

Vamos usar **generators** para tornar isso mais elegante! O decorador `@gen` permite escrever código que **parece síncrono** mas funciona com Effects.

### Inspiração: Async/Await

Esta técnica é **idêntica** ao funcionamento de async/await:

```python
# Async/await
async def fetch_data():
    user = await get_user()      # "Suspende" até completar
    if user.active:
        posts = await get_posts(user.id)  # "Suspende" novamente  
        return posts
    return []

# Generator Effects  
@gen
def fetch_data():
    user = yield get_user_effect()      # "Suspende" até completar
    if user.active:
        posts = yield get_posts_effect(user.id)  # "Suspende" novamente
        return posts
    return []
```

### Como Funciona:

1. **yield**: Pausa execução e retorna Effect para o runner
2. **Runner**: Executa Effect e envia resultado de volta  
3. **send()**: Retoma execução com o resultado
4. **return**: Valor final do Effect

### Vantagens:

- **Sintaxe Natural**: Parece código imperativo normal
- **Controle de Fluxo**: if/else, loops, try/catch funcionam naturalmente  
- **Composição**: Resultado ainda é um Effect normal
- **Debugging**: Mais fácil debuggar que composição funcional pura

Ignorando types porque o sistema de tipos do Python não consegue lidar com essa meta-programação avançada.

### Implementação do Generator Runner

O código a seguir implementa um "runner" que interpreta generators como Effects. É uma versão simplificada do que bibliotecas como `asyncio` fazem internamente:

- **next()**: Inicia o generator e obtém o primeiro Effect
- **send()**: Envia resultado de volta e obtém próximo Effect  
- **StopIteration**: Captura o valor final quando generator termina

Este padrão é tão poderoso que é usado em muitas linguagens modernas (JavaScript, C#, Rust, etc.).

In [117]:
# type: ignore
def gen(generator_func):
    def run_effect():
        generator = generator_func()
        
        try:
            # Start the generator and get the first yielded Effect
            current_effect = next(generator)
            
            while True:
                # Run the yielded Effect to get its result
                result = current_effect()
                
                try:
                    # Send the result back to the generator and get the next Effect
                    current_effect = generator.send(result)
                except StopIteration as e:
                    # Generator completed, return its final value
                    return e.value
                    
        except StopIteration as e:
            # Generator completed without yielding anything
            return e.value
    
    return run_effect

@gen
def my_effect():
    a = yield random_number_function()
    b = yield random_number_function()

    if a > b:
        return f"Primeiro número era maior: {a}"
    else:
        return f"Segundo numero era maior: {b}"

# Ainda é um efeito! Então podemos fazer
run(repeat(my_effect, 2))

['Primeiro número era maior: 92', 'Segundo numero era maior: 35']


## Exemplo Real - Sistema de Notificações Resiliente

Agora vamos aplicar tudo que aprendemos em um cenário real! Vamos construir um sistema de notificações que demonstra como Effects nos ajudam a criar software robusto e observável.

### Características do Sistema:

1. **Múltiplos Canais**: WhatsApp e Email como fallback
2. **Resiliência**: Retry automático em falhas
3. **Observabilidade**: Telemetria completa de operações
4. **Graceful Degradation**: Fallbacks quando APIs estão indisponíveis
5. **Testabilidade**: Fácil de mockar e testar

### Componentes que Vamos Implementar:

1. **Efeito para buscar dados** - Simula API que pode falhar
2. **Telemetria** - Logs de debug, erro e timing
3. **Logs estruturados** - Debug e erro com cores
4. **Enviar notificações** - WhatsApp e Email com simulação de falhas
5. **Retry inteligente** - Tentar novamente apenas se for Exception

### Por que Effects São Ideais Aqui:

- **Composição**: Combinar busca + validação + envio + telemetria
- **Testabilidade**: Mockar APIs sem afetar lógica de negócio
- **Resiliência**: Adicionar retry/fallback de forma declarativa
- **Observabilidade**: Instrumentar sem poluir código de negócio
- **Manutenibilidade**: Cada concern separado em função específica

Este exemplo mostra como Effects não são apenas teoria acadêmica, mas ferramenta prática para sistemas de produção.

In [118]:
from typing import TypedDict, Optional
from time import sleep
from termcolor import colored

emails = [
    "john.doe@example.com",
    "jane.smith@example.com",
    "alice.johnson@example.com",
    "bob.brown@example.com",
    "charlie.davis@example.com",
    "emily.clark@example.com",
    "frank.harris@example.com",
    "grace.lee@example.com",
    "henry.miller@example.com",
    "ivy.wilson@example.com",
]

class User(TypedDict):
    id: int
    whatsapp: Optional[str]
    email: str

def get_user(id: int) -> Effect[User | Exception]:
    # 50% de chance de dar erro na API
    def wrapped_effect():
        if random.random() < 0.5:
            return Exception("Erro ao buscar usuário")
        return User({
            "id": id,
            "whatsapp": random.choice([None, "1234-5678"]),
            "email": emails[id]
        })
    
    return wrapped_effect

def retry_if_exception[A](effect: Effect[A], retries: int) -> Effect[A | Exception]:
    def wrapped_effect():
        for _ in range(retries):
            result = effect()
            if not isinstance(result, Exception):
                return result
        return Exception("Max retries exceeded")
    return wrapped_effect

def send_zap_zap(to: str, message: str) -> Effect[str | Exception]:
    def wrapped_effect():
        if random.random() < 0.5:
            return Exception("Erro ao enviar mensagem pelo WhatsApp")
        sleep(random.random())
        return(f"Zap enviada para {to}")
    return wrapped_effect

def send_email(to: str, subject: str) -> Effect[str | Exception]:
    def wrapped_effect():
        if random.random() < 0.5:
            return Exception("Erro ao enviar email")
        sleep(random.random())
        return(f"Email enviado para {to}")
    return wrapped_effect

def debug_to_gcp[A](effect: Effect[A], message: str) -> Effect[A]: 
    def wrapped_effect():
        result = effect()
        if isinstance(result, Exception):
            print(colored(f"DEBUG: {message}", 'green'))
        return result
    return wrapped_effect

def log_error[A](effect: Effect[A]) -> Effect[A]:
    def wrapped_effect():
        result = effect()
        if isinstance(result, Exception):
            print(colored(f"ERROR: {result}", "red"))
        return result
    return wrapped_effect

def telemetry_effect[A](effect: Effect[A], message: str) -> Effect[A]:
    def wrapped_effect() -> A:
        timed_effect = timed(effect)

        def time_message(pair: tuple[A, float]) -> Effect[A]:
            msg = f"{message}: {pair[1]/10000:.2f} seconds"
            return debug_to_gcp(lambda: pair[0], msg)

        logged = flat_map_effect(timed_effect, time_message)
        err_logged = log_error(logged)
        return err_logged()
    return wrapped_effect


def zap_user(number: str, message: str) -> Effect[str | Exception]:
    return telemetry_effect(send_zap_zap(number, message), f"zap_user: {number}")

def email_user(email: str, subject: str) -> Effect[str | Exception]:
    return telemetry_effect(send_email(email, subject), f"email_user: {email}")


def run_all_effects[A](effects: list[Effect[A]]) -> None:
  results: list[A] = []
  for effect in effects:
      result = effect()
      results.append(result)

  print(results)

## Escrevendo Nossa Main - Programação "Imperativa" com Effects

Agora a mágica acontece! Com o decorador `@gen`, conseguimos escrever código que parece imperativo mas mantém todos os benefícios dos Effects.

### A Transformação Sintática

```python
# Como escrevemos (imperativo)
@gen
def send_notification():
    user = yield get_user_effect()
    if user.active:
        result = yield send_email_effect(user.email)
        return result
    return "User inactive"

# Como o sistema interpreta (funcional)
def send_notification():
    return flat_map_effect(
        get_user_effect(),
        lambda user: send_email_effect(user.email) if user.active 
                    else pure_effect("User inactive")
    )
```

### Força o Olho: yield ≈ await

Agora força o olho e imagina que `yield` na verdade é `await` - **é exatamente a mesma coisa!**

```python
# Generator Effects (nosso código)
user = yield get_user_effect()

# Async/Await (JavaScript/Python)  
user = await get_user_promise()
```

### Vantagens desta Abordagem:

1. **Sintaxe Natural**: Controle de fluxo normal (if/else, loops, try/catch)
2. **Composição Mantida**: Resultado ainda é um Effect composável
3. **Debugging**: Stack traces mais claros
4. **Manutenibilidade**: Código mais legível e fácil de modificar

### Flexibilidade do Sistema:

- **Testabilidade**: Podemos mockar `get_user_effect()` facilmente
- **Telemetria**: Adicionar logging/metrics transparentemente  
- **Retry**: Envolver qualquer Effect com retry
- **Caching**: Adicionar cache sem modificar lógica

Este é o poder real dos Effects: **separar o QUE fazer (lógica) do COMO fazer (execução)**.

In [119]:
# type: ignore

def main(id: int):
      
  @gen
  def generator():
      user = yield retry_if_exception(get_user(id), 4)

      # Aqui podemos utilizar um fluxo normal
      if isinstance(user, Exception):
          return f'Error ao buscar usuário {id}'

      if user["whatsapp"]:
          response = yield retry_if_exception(zap_user(user["whatsapp"], "Olá, você tem uma nova mensagem!"), 2)

      elif user["email"]:
          response = yield retry_if_exception(email_user(user["email"], "Nova mensagem recebida"), 2)

      if isinstance(response, Exception):
          return f"Erro ao enviar mensagem para usuário id {user['id']}"
      
      return response

  return generator

run_all_effects([main(x) for x in range(0,9)])

DEBUG: zap_user: 1234-5678: 0.00 seconds
ERROR: Erro ao enviar mensagem pelo WhatsApp
DEBUG: email_user: bob.brown@example.com: 0.00 seconds
ERROR: Erro ao enviar email
DEBUG: email_user: bob.brown@example.com: 0.00 seconds
ERROR: Erro ao enviar email
DEBUG: zap_user: 1234-5678: 0.00 seconds
ERROR: Erro ao enviar mensagem pelo WhatsApp
DEBUG: email_user: bob.brown@example.com: 0.00 seconds
ERROR: Erro ao enviar email
DEBUG: email_user: bob.brown@example.com: 0.00 seconds
ERROR: Erro ao enviar email
DEBUG: zap_user: 1234-5678: 0.00 seconds
ERROR: Erro ao enviar mensagem pelo WhatsApp
DEBUG: email_user: henry.miller@example.com: 0.00 seconds
ERROR: Erro ao enviar email
DEBUG: email_user: henry.miller@example.com: 0.00 seconds
ERROR: Erro ao enviar email
['Zap enviada para 1234-5678', 'Zap enviada para 1234-5678', 'Error ao buscar usuário 2', 'Erro ao enviar mensagem para usuário id 3', 'Zap enviada para 1234-5678', 'Zap enviada para 1234-5678', 'Zap enviada para 1234-5678', 'Email enviad